# Pipeline Parallelism

In this section we'll go over the basic ideas of model parallelization by placing different layers of the model on different devices.  We'll also briefly cover an example using the under-development and subject-to-change [`torch.distributed.pipelining`](https://docs.pytorch.org/docs/stable/distributed.pipelining.html) API.

## Model Parallelism by Layers

The basic idea of layer-based parallelism is to tackle models larger than what a single GPU can fit by splitting the model into units of layers, and propagating the forward and backward passes through those units:

![Diagram of mmodel parallelism by layer, where a model is broken into three units distributed over three GPUs. A forward pass sweeps through in one direction, followed by a backward pass in the other](images/pipeline.png)

This approach has much to commend it.  In particular, the communication-to-computation ratio can be quite favourable.  The only data that needs to be propagated are the activations (forward pass) or gradients (backward pass) at the boundary of the unit, while the volume of the unit is what needs to be computed on.   This makes it very very well suited for parallelism across nodes; the bandwidth and latency requirements for communication can be quite modest.

![Diagram of layer-based parallelism with and without microbatches.  Without breaking batches up the GPUs are idle much of the time.  By breaking SGD-style minibatches further up into microbatches, the GPU utilization is much improved](images/pipeline-microbatches.png)